# Bayesian classification and clustering

## Naive Bayes Classifier (discrete)

Consider a supervised binary classification problem for discrete vectors $x=(x^i,i=1,...,n)$ given a training set
$$
\{(y_j, x_j), j=1..N\},
$$
where $x_j=(x^i_j,i=1,...,n)$ are given observed vectors with discrete components and $y_j$ are the corresponding binary (or more generally - discrete) classifiction labels. 

In order to predict if a given $x=x^*$ should be classified as $0$ or $1$ (or more generally as one of the finite set of labels) we compare two probabilities
$P(y=1|x=x^*)$ vs $P(y=0|x=x^*)$ defined as a posteriors using Bayes theorem
$$
P(y=b|x)=\frac{P(x|y=b)P(y=b)}{P(x)}
$$
based on the priors $P(y=b)$ and conditional probabilities 
$$
P(x=x^*|y=b)=\prod\limits_{i=1}^n P(x^i={x^*}^i |y=b)
$$
in the "naive" assumption of independence for the components $x^i$. The above probabilities $P(y=b)$ and $P(x^i={x^*}^i |y=b)$ can be derived directly from the training sample as
$$
P(y=b)=\frac{|\{j:y_j=b\}|}{N},
$$
$$
P(x^i={x^*}^i |y=b)=\frac{|\{j:x^i_j={x^*}^i, y_j=b\}|}{|\{j:y_j=b\}|}.
$$

As the denominator $P(x)$ does not depend on $b$ it is enough to compare the numerators, one can see 
$$
P(y=b|x=x^*)\sim P(y=b)\prod\limits_{i=1}^n P(x^i={x^*}^i |y=b),
$$
letting the clasification level to be defined as
$$
y^*={\rm argmax}_{b}P(y=b)\prod\limits_{i=1}^n P(x^i={x^*}^i |y=b)
$$

## Example 1. "Toy" example of spam classification

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import scipy
from scipy import stats
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
import statsmodels.formula.api as smf
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random
#import shapefile

Populating the interactive namespace from numpy and matplotlib


In [2]:
#consider some data about content of several e-mails classified as Spam and non-spam (Ham)
train=pd.DataFrame({'spam':[1, 1, 1, 0, 0, 0],'Lottery':[1, 0, 1, 0, 0, 1],'Cruise':[1, 1, 0, 0, 1, 0],'Win':[1, 1, 0, 1, 0, 0]})
train

,Cruise,Lottery,Win,spam
0,1,1,1,1
1,1,0,1,1
2,0,1,0,1
3,0,0,1,0
4,1,0,0,0
5,0,1,0,0


In [3]:
#Now if we receive emails of the following content - are those spam or not?
test=pd.DataFrame({'Lottery':[1, 0],'Cruise':[1, 0],'Win':[0, 0]})
test

,Cruise,Lottery,Win
0,1,1,0
1,0,0,0


First estimate the prior:
$$
P(spam)=P(ham)=3/6=1/2.
$$
Now estimate the conditional probabilities of having each word (probability of not having it will then be one minus the above probability) if the e-mail is spam or not:
$$
P(Cruise|spam)=2/3
$$
$$
P(Lottery|spam)=2/3
$$
$$
P(Win|spam)=2/3
$$
$$
P(Cruise|ham)=1/3
$$
$$
P(Lottery|ham)=1/3
$$
$$
P(Win|ham)=1/3
$$

Consider first of those two emails containing words "Cruise" and "Lottery" (denote it "{Cruise,Lottery}"). As
$$
P(spam|\{Cruise,Lottery\})\sim P(Cruise|spam)P(Lottery|spam)P(!Win|spam)P(spam)=2/3*2/3*(1-2/3)*1/2=2/27
$$
and
$$
P(ham|\{Cruise,Lottery\})\sim P(Cruise|ham)P(Lottery|ham)P(!Win|ham)P(ham)=1/3*1/3*(1-1/3)*1/2=1/27
$$
we have
$$
P(spam|\{Cruise,Lottery\})>P(ham|\{Cruise,Lottery\})\Rightarrow \{Cruise,Lottery\}\to spam.
$$
So the classifier will mark it as "spam".

For the second e-mail containing none of the words above:
$$
P(spam|none)\sim P(!Cruise|spam)P(!Lottery|spam)P(!Win|spam)P(spam)=1/3*1/3*1/3*1/2=1/54
$$
and
$$
P(ham|none)\sim P(!Cruise|ham)P(!Lottery|ham)P(!Win|ham)P(ham)=2/3*2/3*2/3*1/2=4/27
$$
we have
$$
P(spam|\{Win\})<P(ham|\{Win\})\Rightarrow \{Win\}\to ham.
$$
So the classifier will mark it as "ham".



## Example 2. Taxi trips within and outside Manhattan

Below we apply Naive Bayes Classifier to classying taxi trips. Given a certain sample of workday daytime taxi trips with the information of how long and fast the trip was as well and how many passengers were on it as how big was the tip (all but the passenger count encoded as discrete categorical variables starting from 1 (smallest) and up to 4-6 (highest)) we need to train a classifier of whether the trip happened inside Manhattan (2) or not (1). Trips from  the sample are assigned such labels to serve for the supervised learning purposes as well as for testing the classifier.

In [4]:
#First load the sample and split it randomly into the training (75%) and test (25%) set. 
data = pd.read_csv("data/NYC_taxi_sample.csv",index_col=0)
data.manhattan=data.manhattan-1
random.seed(2015)
ind=stats.bernoulli.rvs(p = 0.75, size = len(data.index))
train=data[ind==1]
test=data[ind==0]
train.head()

,manhattan,tip,dist,speed,pass
26066,0,1,1,1,1
26100,0,3,5,6,1
26354,0,1,6,4,2
26722,0,4,1,1,1
26793,1,3,3,3,2


In [5]:
#how big is the entire sample as well as the training/test sets
[len(data.index),len(train.index),len(test.index)]

[5329, 4061, 1268]

Now implement the estimator for the training sample statistics $P(x=x^*|y=b)$ and $P(y=b)$.

In [6]:
def trainNaiveBayesDiscrete(trainData):
  #training discrete Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
  #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
  dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
  #split the training data between two labels
  ind1=tY==0
  ind2=tY==1
  #estimate P(y=b)  
  dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
  dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
  #estimate conditional probabilities P(x|y=b)
  for j in trainData.columns[1:]:
    for i in range(1,m+1):
        dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
        dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
  return dp

In [7]:
def classifyNaiveBayesDiscrete(classData,dp):
  #classifying using trained discrete Naive Bayes Classifier
  Y=classData[classData.columns[0]]*0 #initialize the empty array 
  for i in classData.index: #for al records to classify
    #start with the priors
    P1=dp[0][classData.columns[0]][1]; 
    P2=dp[1][classData.columns[0]][1];
    #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
    for j in classData.columns[1:]:
      P1=P1*dp[0][j][classData[j][i]]
      P2=P2*dp[1][j][classData[j][i]]
    Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
  return Y

In [8]:
#now train the classifier based on the training sample
dp=trainNaiveBayesDiscrete(train)

In [9]:
dp

[   manhattan         tip       dist      speed        pass
 1  0.5109579   0.5595181  0.2072289  0.1219277   0.7036145
 2        NaN  0.09686747  0.1918072  0.2313253   0.1306024
 3        NaN  0.09590361       0.12   0.206747  0.05686747
 4        NaN   0.1609639  0.1103614  0.1966265   0.1089157
 5        NaN  0.05156627  0.1484337  0.1281928           0
 6        NaN  0.03518072  0.2221687  0.1151807           0,
    manhattan         tip        dist       speed        pass
 1  0.4890421   0.3726083   0.4204431    0.306143   0.6460222
 2        NaN   0.1717019   0.2749245   0.4093656    0.193857
 3        NaN   0.1671702   0.1409869   0.1621349  0.09566969
 4        NaN   0.1696878  0.09415911  0.07200403  0.06445116
 5        NaN  0.05740181  0.02517623  0.03776435           0
 6        NaN  0.06143001  0.04431017  0.01258812           0]

In [10]:
#run the classifier over the test sample
C=classifyNaiveBayesDiscrete(test,dp)

In [11]:
#compute the classification accuracy over the test set comparing the classification obtained with the labels provided
1.0*sum(C==test.manhattan)/len(C)

0.7255520504731862

In [12]:
#also compute the accuracy over the training set
Ct=classifyNaiveBayesDiscrete(train,dp)
1.0*sum(Ct==train.manhattan)/len(Ct)

0.7049987687761635

# Naive Bayes Classifier (Gaussian)

But in many cases the features are arbitrary real-valued (continous) numbers. As before consider a supervised binary classification problem for real-valued vectors $x=(x^i,i=1,...,n)$ given a training set
$$
\{(y_j, x_j), j=1..N\},
$$
where $x_j=(x^i_j,i=1,...,n)$ are given observed vectors with real-valued components and $y_j$ are the corresponding discrete classifiction labels. 

Similarly to the discrete framework one can see
$$
P(y=b|x=x^*)\sim p(x=x^*|y=b)P(y=b)=P(y=b)\prod\limits_{i=1}^n p(x^i={x^*}^i |y=b).
$$

However in such a case the above framework won't work exactly the same way as we can not directly estimate all the probabilities $P(x^i={x^*}^i|y=b)$. In this case we need to fit a certain continous probability distributions for them, estimating probability densities $p(x^i={x^*}^i|y=b)$ rather than probabilities. In this section we'll present a framework assuming normal (Gaussian) distributions:
$$
p(x^i={x^*}^i|y=b)=\frac{1}{\sqrt{2\pi}\sigma_{i,b}}e^{-\frac{({x^*}^i-\mu_{i,b})^2}{2\sigma_{i,b}^2}},
$$
where parameters $\mu_{i,b}$ and $\sigma_{i,b}$ can be simply defined (those willing to have a rigorious probabilistic derivation can try estimating those parameters through max-likelihood approach, which gives the same result) as sample mean and standard deviation:
$$
\mu_{i,b}=\frac{\sum\limits_{j:x^i_j={x^*}^i, y_j=b}x^i_j}{|\{j:x^i_j={x^*}^i, y_j=b\}|},
$$
$$
\sigma_{i,b}=\sqrt{\frac{\sum\limits_{j:x^i_j={x^*}^i, y_j=b}(x^i_j-\mu_{i,b})^2}{|\{j:x^i_j={x^*}^i, y_j=b\}|}}.
$$

The probabilities $P(y=b)$ can be defined as before:
$$
P(y=b)=\frac{|\{j:y_j=b\}|}{N}.
$$

Then
$$
P(y=b|x=x^*)\sim p(x=x^*|y=b)P(y=b)=P(y=b)\prod\limits_{i=1}^n p(x^i={x^*}^i |y=b)\sim
$$$$
\sim P(y=b)e^{-\sum\limits_{i=1}^n \frac{({x^*}^i-\mu_{i,b})^2}{2\sigma_{i,b}^2}}.
$$
Finally the clasification level can be defined as
$$
y^*={\rm argmax}_{b}\left[ln(P(y=b))-\sum\limits_{i=1}^n\frac{({x^*}^i-\mu_{i,b})^2}{2\sigma_{i,b}^2}\right]
$$

In [13]:
def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  #estimate priors
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  #estimate sample distribution paramters for p(xi|y=b)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

In [14]:
def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    #start from the priors
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu2'];
    #multiply by conditional probability densities p(xi|y=b)
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

# Example 3. Predicting type of the house

Based on the sample of characteristics and the prices of the single unit residential and commercial houses sold in "our" zip code 11201 in downtown Brooklyn between the years 2009 and 2012, build a classifier defining if the sold house was actually residential or commercial.

In [15]:
#read the sample
data = pd.read_csv("data/NYC_RE_sample.csv")
# split sample into training and test ones
data.columns=['index','area','land','year','price','bldtype'] #house size, land, year built, price and type
data.bldtype=data.bldtype-1 #bring the house classes to 0 - residential, 1 - commercial
data.head()

,index,area,land,year,price,bldtype
0,1,2607,1200,2010,825000,0
1,2,1950,1783,1899,1685000,0
2,3,2520,1875,1899,1100000,0
3,4,3750,3125,1931,1200000,1
4,5,7812,5021,1908,1900000,1


In [16]:
#transform the features to relative values getting rid of obvious correlations: consider price per sq. foot, land per sq. foot of indoor area
data1=pd.DataFrame({'Y':data.bldtype,'price':data.price/data.area,'land':data.land/data.area,'year':data.year})

In [17]:
#split dataset into 60% training and 40% test 
np.random.seed(2015)
ind=stats.bernoulli.rvs(p = 0.6, size = len(data.index))
train=data1[ind==1]
test=data1[ind==0]
train.head()

,Y,land,price,year
1,0,0.914359,864.102564,1899
3,1,0.833333,320.000000,1931
6,0,0.666966,1049.160671,1901
8,1,0.200000,340.000000,1900
10,0,0.510204,510.204082,1901


In [18]:
dp=trainNaiveBayes(train) #train the Gausian Naive Bayes Classifier

In [19]:
dp

,Y,land,price,year
mu1,0.6792453,0.6391921,1010.221,1907.139
sigma1,NaN,0.2639759,388.859,46.07333
mu2,0.3207547,0.9142577,384.538,1928.529
sigma2,NaN,0.9183005,264.8162,21.81203


In [20]:
C=classifyNaiveBayes(test,dp) #classify test set

In [21]:
#classification accuracy (over test set)
100.0*sum(C==test.Y)/len(C)

82.92682926829268

In [22]:
#also report accuracy of the classifier over the training - it is slightly higher, although not that higher
Ct=classifyNaiveBayes(train,dp)
100.0*sum(Ct==train.Y)/len(Ct)

88.67924528301887

# Example 4. Spam classification

1. Title:  SPAM E-mail Database

2. Sources:
   (a) Creators: Mark Hopkins, Erik Reeber, George Forman, Jaap Suermondt
        Hewlett-Packard Labs, 1501 Page Mill Rd., Palo Alto, CA 94304
   (b) Donor: George Forman (gforman at nospam hpl.hp.com)  650-857-7835
   (c) Generated: June-July 1999

3. Past Usage:
   (a) Hewlett-Packard Internal-only Technical Report. External forthcoming.
   (b) Determine whether a given email is spam or not.
   (c) ~7% misclassification error.
       False positives (marking good mail as spam) are very undesirable.
       If we insist on zero false positives in the training/testing set,
       20-25% of the spam passed through the filter.

4. Relevant Information:
        The "spam" concept is diverse: advertisements for products/web
        sites, make money fast schemes, chain letters, pornography...
	The collection of spam e-mails came from the postmaster and 
	individuals who had filed spam.  The collection of non-spam 
	e-mails came from filed work and personal e-mails, and hence
	the word 'george' and the area code '650' are indicators of 
	non-spam.  These are useful when constructing a personalized 
	spam filter.  One would either have to blind such non-spam 
	indicators or get a very wide collection of non-spam to 
	generate a general purpose spam filter.

        For background on spam:
        Cranor, Lorrie F., LaMacchia, Brian A.  Spam! 
        Communications of the ACM, 41(8):74-83, 1998.

5. Number of Instances: 4601 (1813 Spam = 39.4%)

6. Number of Attributes: 58 (57 continuous, 1 nominal class label)

7. Attribute Information:
The last column of 'spambase.data' denotes whether the e-mail was 
considered spam (1) or not (0), i.e. unsolicited commercial e-mail.  
Most of the attributes indicate whether a particular word or
character was frequently occuring in the e-mail.  The run-length
attributes (55-57) measure the length of sequences of consecutive 
capital letters.  For the statistical measures of each attribute, 
see the end of this file.  Here are the definitions of the attributes:

48 continuous real [0,100] attributes of type word_freq_WORD 
= percentage of words in the e-mail that match WORD,
i.e. 100 * (number of times the WORD appears in the e-mail) / 
total number of words in e-mail.  A "word" in this case is any 
string of alphanumeric characters bounded by non-alphanumeric 
characters or end-of-string.

6 continuous real [0,100] attributes of type char_freq_CHAR
= percentage of characters in the e-mail that match CHAR,
i.e. 100 * (number of CHAR occurences) / total characters in e-mail

1 continuous real [1,...] attribute of type capital_run_length_average
= average length of uninterrupted sequences of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_longest
= length of longest uninterrupted sequence of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_total
= sum of length of uninterrupted sequences of capital letters
= total number of capital letters in the e-mail

1 nominal {0,1} class attribute of type spam
= denotes whether the e-mail was considered spam (1) or not (0), 
i.e. unsolicited commercial e-mail.  


8. Missing Attribute Values: None

9. Class Distribution:
	Spam	  1813  (39.4%)
	Non-Spam  2788  (60.6%)



In [23]:
import urllib
data = urllib.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data").read()
data_name=urllib.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.names").read()
# Read the data
data=data.split("\r\n")
data_spam=[]
for i in range(len(data)):
    if len(data[i])>0:
        temp=data[i].split(",")
        #change from str to float
        t_l=[]
        for j in range(len(temp)):
            t_l.append(float(temp[j]))
        data_spam.append(t_l)

#Read the column names
temp=data_name.split("\r\n")
column_names=[]
for i in temp:
    if (i.startswith('word') or i.startswith('char') or i.startswith('capital')):
        column_names.append(i.split(":")[0])
column_names.append("spam") 

In [24]:
#Generate the data set for our tests
data_spam=pd.DataFrame(data_spam)
data_spam.columns=column_names
X=data_spam.iloc[:,0:-1]
y=data_spam.iloc[:,-1]
data_spam=pd.concat([y,X], axis=1)
data_spam.head()

,spam,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,1,0.00,0.64,0.64,0,0.32,0.00,0.00,0.00,0.00,...,0,0.00,0.000,0,0.778,0.000,0.000,3.756,61,278
1,1,0.21,0.28,0.50,0,0.14,0.28,0.21,0.07,0.00,...,0,0.00,0.132,0,0.372,0.180,0.048,5.114,101,1028
2,1,0.06,0.00,0.71,0,1.23,0.19,0.19,0.12,0.64,...,0,0.01,0.143,0,0.276,0.184,0.010,9.821,485,2259
3,1,0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,...,0,0.00,0.137,0,0.137,0.000,0.000,3.537,40,191
4,1,0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,...,0,0.00,0.135,0,0.135,0.000,0.000,3.537,40,191


In [25]:
# Divide the data into test and traing sets
random.seed(2015)
train_index=random.sample(list(range(len(X))),int(len(X)*0.8))
test_index=[x for x in list(range(len(X))) if x not in train_index]
X_train=X.iloc[train_index,:]
X_test=X.iloc[test_index,:]
y_train=y.iloc[train_index]
y_test=y.iloc[test_index]

In [26]:
traindata=pd.concat([y_train,X_train],axis=1)
dp=trainNaiveBayes(traindata)
dp

,spam,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
mu1,0.6089674,0.06809014,0.2271709,0.2000892,0.0009638554,0.1855734,0.04554217,0.007920571,0.03915663,0.03802767,...,0.04741633,0.05046006,0.1602102,0.020888,0.1164801,0.01184293,0.02108121,2.3761,18.32664,159.0553
sigma1,NaN,0.2657866,1.547869,0.5011758,0.02287893,0.6263632,0.2278229,0.08464661,0.2492644,0.2054897,...,0.3397216,0.3054744,0.2468942,0.1126406,0.9015804,0.07218688,0.2269393,5.533877,41.44568,337.002
mu2,0.3910326,0.1588047,0.16836,0.3985615,0.1753231,0.5122307,0.1727728,0.2825295,0.2103127,0.1691522,...,0.002182071,0.02077137,0.1091543,0.007288395,0.5123878,0.1685726,0.08813899,9.962755,107.4295,475.5949
sigma2,NaN,0.3236926,0.3616037,0.4681438,2.400083,0.7161839,0.3137195,0.605078,0.5659269,0.3532787,...,0.02726459,0.09269251,0.3088427,0.04689625,0.7348309,0.3257907,0.6805679,53.7536,324.751,867.4377


In [27]:
#OS Test
testdata=pd.concat([y_test,X_test],axis=1)
C=classifyNaiveBayes(testdata,dp)

In [28]:
print "we classified e-mails correctly in {0} percents of cases".format(100.0*sum(C==y_test)/len(y_test)) 

we classified e-mails correctly in 80.6731813246 percents of cases


In [29]:
print "But among 20% of errors, {0} percent are ham classified as spam".format(100.0*sum((C==1)&(y_test==0))/len(y_test))

But among 20% of errors, 16.2866449511 percent are ham classified as spam


In [30]:
print "As a result {0} percent of spam e-mails are detected".format(100.0*sum((C==1)&(y_test==1))/sum(y_test==1))

As a result 92.513368984 percent of spam e-mails are detected


In [31]:
print "But {0} percent of non-spam e-mails got classified as spam".format(100.0*sum((C==1)&(y_test==0))/sum(y_test==0))

But 27.4223034735 percent of non-spam e-mails got classified as spam


Clearly the price of two different types of error - a) false negatives, i.e. missed spam or b) false positives, i.e. ham classified as spam - is quite different. In the first case we only get certain inconvenience of still getting some spam e-mails missed by our classifier, but in the second case we can actually miss important information from ham e-mails classified as spam.

So in this specific case the false positives should be additionally penalized. Or we can set a threshold for our level of tolerance to false positives - say we allow no more than 1% of ham be classified as spam. The way to adjust our classifier is to let it classify the e-mail as "spam" not just whenever it seems more likely to be spam than ham, i.e. when $P(spam|x)> P(ham|x)$, but only when we're strongly confident, i.e. when 
$$
P(spam|x)>\lambda P(ham|x),
$$
where $\lambda$ is a selected constant, higher than $1$. The higher is $\lambda$ the stronger is the criteria for spam classification and the lower is the tolerance to false positives. As those posterior probabilities $P(spam|x)$ and $P(ham|x)$ are proportional to the priors $P(spam)$, $P(ham)$, the other equivalent way to implement such an adjustment rather than by introducing a constant $\lambda$ on the decision step, is through modifying the prior as $P(spam|x):=P(spam|x)/\alpha$, where $\alpha>1$. This means that we deliberately lower our prior belief that the e-mail is spam making it harder for the classifier to alter this opinion.

In practice, selection of the constant $\lambda$ or $\alpha$ could be performed in an empiric way once validation sample is separated from the training one - we keep increasing it until the rate of false positives over the validation sample becomes satisfactory.

In [32]:
# Divide the training data into new training and validation sets
random.seed(2015)
train_index1=random.sample(list(range(len(y_train))),int(len(y_train)*0.7))
valid_index=[x for x in list(range(len(y_train))) if x not in train_index1]
traindata1=traindata.loc[train_index1]
validdata=traindata.loc[valid_index]

In [33]:
dpl=trainNaiveBayes(traindata1)
C=classifyNaiveBayes(testdata,dpl)
fp=100.0*sum((C==1)&(testdata.spam==0))/sum(testdata.spam==0)
acc=format(100.0*sum(C==testdata.spam)/len(testdata.spam))
print "Initially we classified e-mails correctly in {0} percents of cases with {1} percent of ham classified as spam".format(acc,fp)     
while True:
    dpl.spam[2]=dpl.spam[2]/1000 #lower the prior P(spam) - need to low it very quickly as due to the high amount of regressors only really low prior would have a major impact on the classifier
    dpl.spam[0]=1-dpl.spam[2]
    C=classifyNaiveBayes(validdata,dpl)
    fp=100.0*sum((C==1)&(validdata.spam==0))/sum(validdata.spam==0)
    print "False positives = {0} percent".format(fp)
    if fp<3: #once the percentage of false positives goes below 3%
         break
C=classifyNaiveBayes(testdata,dpl)
fp=100.0*sum((C==1)&(testdata.spam==0))/sum(testdata.spam==0)
acc=format(100.0*sum(C==testdata.spam)/len(testdata.spam))
print "We classified e-mails correctly in {0} percents of cases with only {1} percent of ham classified as spam".format(acc,fp)     

Initially we classified e-mails correctly in 82.4104234528 percents of cases with 24.1316270567 percent of ham classified as spam
False positives = 13.6771300448 percent
False positives = 6.05381165919 percent
False positives = 4.03587443946 percent
False positives = 3.13901345291 percent
False positives = 2.69058295964 percent
We classified e-mails correctly in 85.016286645 percents of cases with only 4.38756855576 percent of ham classified as spam


## Semi-supervised EM classifier

This is typically done based on the Expectation Maximization (EM) algorithm. As before assume we have a set of observations for the input variable vectors $x=(x^i,i=1,...,n)$ as well as the corresponding labels (discrete output variable $y$)
$$
\{(y_j, x_j), j=1..N\},
$$
where $x_j=(x^i_j,i=1,...,n)$ are given observed vectors of discrete features and $y_j$ are the corresponding classifiction labels. However suppose that some of the $y_j$ are not known and let $y_j=nan$ in this case.

The algorithm:

Step 1. Based on the labeled subset of the training data (i.e. having $y_j\neq nan$) estimate sample conditional probabilities $\theta_{b,i,a}^0=P(x^i=a|y=b)$ as well as sample prior probabilities $\theta_b^0=p(y=b)$ for each of the possible values $b$ of $y$ and $a$ of $x^i$. Set $t=0$.

Step 2. Given the current estimate $\theta=\theta^t$ for $P(x^i|y=b)$ and $P(y=b)$ define the unobserved labels $y_j$ as the discrete random variables $\hat{y}_j$ with the probability distribution:
$$
P(\hat{y}_j=b|x_j,\theta^t)=\frac{P(y=b)\prod\limits_{i=1}^n P(x^i_j |y=b)}{\sum_c P(y=c)\prod\limits_{i=1}^n P(x^i_j |y=c)}=\frac{\theta_b^t\prod\limits_{i=1}^n \theta_{b,i,x^i_j}^t}{\sum_c \theta_c^t\prod\limits_{i=1}^n \theta_{c,i,x^i_j}^t}.
$$
In case the label $y_j$ is observed we simply let
$$
P(\hat{y}_j=b|x_j,\theta^t)=\left\{\begin{array}{c}1, b=y_j,\\0, b\neq y_j.\end{array} \right.
$$

Step 3. Re-estimate the parameters $\theta=\theta^{t+1}$ (maximizing the likelihood of the actual observations, see below) for the distributions of $P(x^i|y=b)$ as well as $P(y=b)$ given the label probabilistic estimates with respect to the probabilities $P(\hat{y}_j=b|x_j,\theta^t)$ defined in step 2.

Step 4. If $\theta$ does not change much, i.e. if the termination condition
$$
||\theta^{t+1}-\theta^t||_2=\sum_b (\theta_b^{t+1}-\theta_b^t)^2+\sum_{k,i,a} (\theta_{k,i,a}^{t+1}-\theta_{k,i,a}^t)^2<\varepsilon
$$ 
holds - stop with a final estimate $\theta=\theta^{t+1}$, otherwise set $t:=t+1$ and repeat from step 2.

An important component of the algorithm to be clarified is estimating the parameters $\theta^{t+1}$. It is based on a max-likelihood approach, which can be shown to be a streightforward generalization of the sample probability estimates with respect to the label estimate uncertainty. Specifically one could let:
$$
\theta_b^{t+1}=P(y=b)=\frac{\sum\limits_j P(\hat{y}_j=b|x^j,\theta^t)}{|\{j\}|}
$$
and
$$
\theta_{b,i,a}^{t+1}=P(x_i=a|y=b)=\frac{\sum\limits_{j,x_i^j=a} P(\hat{y}_j=b|x^j,\theta^t)}{\sum\limits_j P(\hat{y}_j=b|x^j,\theta^t)}.
$$

At each step those choices of $\theta$ are intended to maximize the log-likelihood of observing all the $x^j_i$ we are given, which can be defined as
$$
L(\theta)=\sum\limits_j log\left(\sum\limits_b P(y=b)\prod\limits_i P(x_i=x^j_i|y=b)\right)=\sum\limits_j log\left(\sum\limits_b \theta_b\prod\limits_i \theta_{b,i,x^i_j}\right).
$$

Finally, once the iterative EM algorithm helps to infer $\theta$, the classification of new any observations $x^*=\{{x^*}^i\}$ is pretty streightforward and goes pretty much along the lines of the Naive Bayes classifier:
$$
y^*={\rm argmax}_{b}P(y=b)\prod\limits_{i=1}^n P(x^i={x^*}^i |y=b)={\rm argmax}_{b}\theta_b\prod\limits_{i=1}^n \theta_{b,i,{x^i}^*}.
$$

# Example 5. Taxi trip classification with partially missing labels

In [34]:
#read the same data for the taxi trip as used before
data = pd.read_csv("data/NYC_taxi_sample.csv",index_col=0)
data.manhattan=2-data.manhattan
#data structure: categorical variables showing if the trip was within Mangattan and how big/small were the distance travelled, speed, tip left as well as how many passengers were on the trip
data.head()

,manhattan,tip,dist,speed,pass
26066,1,1,1,1,1
26100,1,3,5,6,1
26354,1,1,6,4,2
26589,0,2,2,2,1
26722,1,4,1,1,1


In [35]:
#as before take approximately 75% of the dataset for the training set, reserving the rest for the test set
np.random.seed(2014)
ind=stats.bernoulli.rvs(p = 0.75, size = len(data.index))
train=data[ind==1]
test=data[ind==0]
train.head()

,manhattan,tip,dist,speed,pass
26066,1,1,1,1,1
26100,1,3,5,6,1
26354,1,1,6,4,2
26722,1,4,1,1,1
26793,0,3,3,3,2


In [36]:
#split output variable from the features
X=train.iloc[:,1:]
y=train.iloc[:,0]
X_test=test.iloc[:,1:]
y_test=test.iloc[:,0]

In [37]:
#now let's randomly delete 99.5% of the labels from the training set
random.seed(2015)
Label_index=random.sample(list(range(len(X))),int(len(X)*0.005))
Unlabel_index=[x for x in list(range(len(X))) if x not in Label_index]

X_Label=X.iloc[Label_index,:]
X_Unlabel=X.iloc[Unlabel_index,:]   
y_Label=y.iloc[Label_index]
y_Unlabel=y.iloc[Unlabel_index]# we should not use this information

Let's see what happens if we rely only on the labeled data training Naive Bayes Classifier over it

In [38]:
trainData=pd.concat([y_Label,X_Label],axis=1)
len(y_Label)

19

In [39]:
dp=trainNaiveBayesDiscrete(trainData)
dp

[   manhattan         tip        dist       speed        pass
 1  0.7368421         0.5   0.1428571   0.1428571   0.5714286
 2        NaN  0.07142857   0.3571429   0.4285714   0.2142857
 3        NaN   0.2142857   0.2142857  0.07142857   0.1428571
 4        NaN  0.07142857   0.2142857   0.2857143  0.07142857
 5        NaN           0           0           0           0
 6        NaN   0.1428571  0.07142857  0.07142857           0,
    manhattan  tip dist speed pass
 1  0.2631579  0.6  0.2   0.2  0.8
 2        NaN    0    0   0.2    0
 3        NaN    0  0.2   0.4    0
 4        NaN  0.2  0.2     0  0.2
 5        NaN    0  0.2     0    0
 6        NaN  0.2  0.2   0.2    0]

In [40]:
testdata=pd.concat([y_test,X_test],axis=1)
C=classifyNaiveBayesDiscrete(testdata,dp)

In [41]:
acc=format(100.0*sum(C==y_test)/len(y_test))
print "We correctly classified {0} percents of the trips based on the labeled data only".format(acc)     

We correctly classified 50.8148148148 percents of the trips based on the labeled data only


In [42]:
#implementation of Expectation-Maximization algorithm for partially labeled data
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    
    if (sum(sum((dp1[0]-dp[0])**2))+sum(sum((dp1[1]-dp[1])**2)))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels:    
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
            L=L+math.log(P)
        
        print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
  return dp



In [43]:
#perform EM estimation for theta
dpEM=EM(X_Label,y_Label,X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(testdata,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y_test)/len(y_test)
print "After EM we correctly classified {0} percents of the trips".format(acc)

Iteration 1: log maximum liklihood = -23104.910287
Iteration 2: log maximum liklihood = -22956.539844
Iteration 3: log maximum liklihood = -22733.3814326
Iteration 4: log maximum liklihood = -22418.4482997
Iteration 5: log maximum liklihood = -22144.6817837
Iteration 6: log maximum liklihood = -22030.6757311
Iteration 7: log maximum liklihood = -21997.1619209
Iteration 8: log maximum liklihood = -21981.4247846
After EM we correctly classified 67.2592592593 percents of the trips


# Unsupervised EM clustering

As one can see from the semi-supervised EM framework, the observed portion of the labels have a relatively low impact on the final estimate - they only participate in the initialization of $\theta^0$ and further override the uncertainty for the estimates $\hat{y}_j$ (while those uncertain estimates can actually be available for the observed labels in the same way as for the unobserved ones, already without taking into account the original values $y_j$).

So what if none of the labels $y_j$ are observed? What it changes in the above framework is only the initialization step - as now we do not have any reasonable guess for $\theta^0$ we simply start from a random or homogeneous choice of those probabilities, provided that the probabilities are notmalized appropriately, i.e. $\sum_k \theta_k^0=1$ and $\sum_a \theta_{k,i,a}^0=1$. Once $\theta^0$ is selected the steps 2-4 of the algorithm are exactly the same as before (just having all the $y_j=nan$, i.e. no estimates $\hat{y}_j$ to fix).

This way EM algorithm then enables clustering for the observations $x_j$, inferring the underlying probabilities $\theta$, unobserved labels $y_j$ and training a classifier for any future observations $x^*$ as before:
$$
y^*={\rm argmax}_{b}P(y=b)\prod\limits_{i=1}^n P(x^i={x^*}^i |y=b)={\rm argmax}_{b}\theta_b\prod\limits_{i=1}^n \theta_{b,i,{x^i}^*}.
$$

# Example 6. Taxi trip classification with no labels


In [44]:
X_Unlabel=X #take all observations unlabeled

In [45]:
np.random.seed(2016)
#initialize theta randomly
dp[0].iloc[0,0]=np.random.uniform(0,1)
dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
for j in range(1,len(dp[0].T)):
    b=np.random.uniform(0,1,len(dp[0]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[0].iloc[:,j]=b
for j in range(1,len(dp[1].T)):
    b=np.random.uniform(0,1,len(dp[1]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[1].iloc[:,j]=b


In [46]:
dp

[   manhattan       tip      dist     speed      pass
 1  0.8967054  0.203711  0.233433  0.052900  0.275895
 2        NaN  0.218506  0.246143  0.221832  0.275194
 3        NaN  0.206894  0.205944  0.216376  0.020212
 4        NaN  0.128907  0.191068  0.025485  0.103416
 5        NaN  0.179253  0.079939  0.346481  0.109967
 6        NaN  0.062729  0.043473  0.136927  0.215316,
    manhattan       tip      dist     speed      pass
 1  0.1032946  0.145768  0.248003  0.008330  0.138931
 2        NaN  0.118057  0.187649  0.327176  0.128262
 3        NaN  0.159788  0.189117  0.167362  0.298924
 4        NaN  0.346520  0.101663  0.130922  0.113730
 5        NaN  0.161480  0.147634  0.289818  0.229165
 6        NaN  0.068387  0.125933  0.076393  0.090988]

In [47]:
#perform EM estimation for theta
dpEM=EM([],[],X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(testdata,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y_test)/len(y_test)
if acc<50: #if most of the labels are misclassified it is just the matter of swapping the labels 1<->0
    acc=100-acc
print "After EM we correctly classified {0} percents of the trips".format(acc)

Iteration 1: log maximum liklihood = -23151.4496838
Iteration 2: log maximum liklihood = -23102.1686162
Iteration 3: log maximum liklihood = -23066.4449969
Iteration 4: log maximum liklihood = -23038.4978789
Iteration 5: log maximum liklihood = -23007.8977578
Iteration 6: log maximum liklihood = -22947.8175748
Iteration 7: log maximum liklihood = -22799.4324681
Iteration 8: log maximum liklihood = -22511.6032021
Iteration 9: log maximum liklihood = -22203.7134995
Iteration 10: log maximum liklihood = -22039.6582528
Iteration 11: log maximum liklihood = -21981.8045475
Iteration 12: log maximum liklihood = -21957.5491092
Iteration 13: log maximum liklihood = -21943.1487191
After EM we correctly classified 67.7037037037 percents of the trips
